<a href="https://colab.research.google.com/github/bm23abb/7PAM2002-0509-2023---Data-Science-Project/blob/main/7PAM2002_0509_2023_Data_Science_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import BayesianRidge
from tabulate import tabulate

In [4]:
file_path = '/content/drive/MyDrive/7PAM2002-0509-2023-Data-Science-Project/Data-Science-Project/Bitcoin.csv'
data_frame = pd.read_csv(file_path)

In [15]:
print("First few rows of the dataset:")
print(tabulate(data_frame.head(), headers='keys', tablefmt='grid'))

First few rows of the dataset:
+---------------------+---------+---------+---------+---------+-------------+-------------+--------+---------+-------+-------------+
| Date                |    Open |    High |     Low |   Close |   Adj Close |      Volume |   Year |   Month |   Day |   DayOfWeek |
+=====================+=========+=========+=========+=========+=============+=============+========+=========+=======+=============+
| 2014-09-17 00:00:00 | 465.864 | 468.174 | 452.422 | 457.334 |     457.334 | 2.10568e+07 |   2014 |       9 |    17 |           2 |
+---------------------+---------+---------+---------+---------+-------------+-------------+--------+---------+-------+-------------+
| 2014-09-18 00:00:00 | 456.86  | 456.86  | 413.104 | 424.44  |     424.44  | 3.44832e+07 |   2014 |       9 |    18 |           3 |
+---------------------+---------+---------+---------+---------+-------------+-------------+--------+---------+-------+-------------+
| 2014-09-19 00:00:00 | 424.103 | 427.

In [11]:
data_frame['Date'] = pd.to_datetime(data_frame['Date'])
data_frame.set_index('Date', inplace = True)
data_frame = data_frame.sort_values('Date')

data_frame = data_frame.fillna(method = 'ffill')

data_frame['Year'] = data_frame.index.year
data_frame['Month'] = data_frame.index.month
data_frame['Day'] = data_frame.index.day
data_frame['DayOfWeek'] = data_frame.index.dayofweek

pforp = data_frame['Close'].values.reshape(-1,1)

In [17]:
print("First few rows of the preprocessed dataset:")
print(tabulate(data_frame.head(), headers='keys', tablefmt='grid'))

First few rows of the preprocessed dataset:
+---------------------+---------+---------+---------+---------+-------------+-------------+--------+---------+-------+-------------+
| Date                |    Open |    High |     Low |   Close |   Adj Close |      Volume |   Year |   Month |   Day |   DayOfWeek |
+=====================+=========+=========+=========+=========+=============+=============+========+=========+=======+=============+
| 2014-09-17 00:00:00 | 465.864 | 468.174 | 452.422 | 457.334 |     457.334 | 2.10568e+07 |   2014 |       9 |    17 |           2 |
+---------------------+---------+---------+---------+---------+-------------+-------------+--------+---------+-------+-------------+
| 2014-09-18 00:00:00 | 456.86  | 456.86  | 413.104 | 424.44  |     424.44  | 3.44832e+07 |   2014 |       9 |    18 |           3 |
+---------------------+---------+---------+---------+---------+-------------+-------------+--------+---------+-------+-------------+
| 2014-09-19 00:00:00 | 4

In [18]:
normalize = MinMaxScaler(feature_range=(0 ,1))
normalize_price = normalize.fit_transform(pforp)

In [19]:
n_p_df = pd.DataFrame(normalize_price, columns=['Normalize close'])
print("First few rows of Min-Max Normalize prices:")
print(tabulate(n_p_df.head(), headers='keys', tablefmt="grid"))

First few rows of Min-Max Normalize prices:
+----+-------------------+
|    |   Normalize close |
+====+===================+
|  0 |        0.00414359 |
+----+-------------------+
|  1 |        0.00365546 |
+----+-------------------+
|  2 |        0.00321557 |
+----+-------------------+
|  3 |        0.00342492 |
+----+-------------------+
|  4 |        0.0032753  |
+----+-------------------+


In [20]:
def create_datset(dataset, look_back=60):
  x, y = [], []
  for i in range(len(dataset) - look_back - 1):
    a = dataset[i:(i + look_back), 0]
    x.append(a)
    y.append(dataset[i + look_back, 0])
  return np.array(x), np.array(y)

look_back = 60
x, y = create_datset(normalize_price, look_back)
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))